In [ ]:
# Import libraries
import os
import sys
import subprocess
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from pyfonts import set_default_font, load_google_font
import cmcrameri.cm as cmc

# Find GRASS Python packages
sys.path.append(
  subprocess.check_output(
    ["grass", "--config", "python_path"],
    text=True
    ).strip()
  )

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj
from grass.tools import Tools
import grass.script.array as ga

# Create a temporary folder
import tempfile
temporary = tempfile.TemporaryDirectory()

# Create a project in the temporary directory
gs.create_project(path=temporary.name, name="noise")

# Start GRASS in this project
session = gj.init(Path(temporary.name, "noise"))
tools = Tools()

In [ ]:
# Set region
rows = 200
cols = 800
resolution = 1
tools.g_region(s=0, w=0, n=rows * resolution, e=cols * resolution, res=resolution)

In [ ]:
# import libraries
import random

# Set parameters
seed = 0
amplitude = 100.0
iterations = 3
wavelength = 33
octaves = 6
lacunarity = 0.5
gain = 0.2

# Generate zeros
tools.r_mapcalc(expression="fbm = 0")

# Generate fractional Brownian motion
for octave in range(octaves):

    # Generate random surface
    tools.r_mapcalc(
        expression=f"noise = rand(-{amplitude}, {amplitude})",
        seed=seed
        )
    
    # Generate gradient noise
    for i in range(iterations):
    
        # Smooth noise
        tools.r_neighbors(
          input="noise",
          output="noise",
          size=wavelength,
          method="average",
          flags="c",
          overwrite=True
          )

    # Calculate sum of gradient noise maps
    tools.r_mapcalc(expression="fbm = fbm + noise")

    # Increment parameters
    amplitude = amplitude * gain
    wavelength = round(wavelength * lacunarity)
    if wavelength % 2 == 0:
        wavelength += 1

# Visualize
m = gj.Map(width=800)
m.d_rast(map="fbm")
m.d_legend(raster="fbm", at=(5, 95, 1, 3))
m.show()

In [ ]:
# Convert to array
fbm = ga.array("fbm")

In [ ]:
# Set font
font = load_google_font("Fira Sans")
set_default_font(font)
plt.rcParams.update({"font.size": 6})

# Plot figure
figure = plt.figure()
ax = figure.add_subplot()
ax.set_xticks([])
ax.set_yticks([])
image = ax.imshow(fbm, cmap=cmc.devon)
legend = figure.colorbar(image, shrink=0.275)
legend.ax.tick_params(size=0)

# Save image
figure.savefig(
    "figure_4.png",
    dpi=600,
    bbox_inches="tight",
    )